In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25


In [ ]:
!pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import os

In [ ]:
from groq import Groq

In [ ]:
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(model_name='llama3-70b-8192')

In [ ]:
llm.invoke('what is nlp')

AIMessage(content="NLP stands for Natural Language Processing, which is a subfield of artificial intelligence (AI) that deals with the interaction between computers and human language. It involves the development of algorithms and statistical models that enable computers to process, understand, and generate natural language data, such as text or speech.\n\nThe goal of NLP is to enable computers to perform tasks that would normally require human-level understanding of language, such as:\n\n1. **Language Translation**: Translating text from one language to another.\n2. **Sentiment Analysis**: Determining the emotional tone or sentiment behind a piece of text, such as whether it's positive, negative, or neutral.\n3. **Text Classification**: Classifying text into categories, such as spam vs. non-spam emails.\n4. **Named Entity Recognition**: Identifying named entities, such as people, places, and organizations, in text.\n5. **Speech Recognition**: Transcribing spoken language into written 

In [ ]:
from langchain_core.tools import tool

In [ ]:
def add(a, b):
  return a + b

def multiply(a, b):
  return a * b

In [ ]:
multiply(1,2)

2

In [ ]:
@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a ** b

In [ ]:
from datetime import datetime
@tool
def today_date():
  """Returns today's date."""
  return datetime.now().strftime('%Y-%m-%d')


In [ ]:
today_date.run({})

'2024-12-24'

In [ ]:
tools = [today_date, add, multiply]

In [ ]:
from langchain import hub

In [ ]:
promt = hub.pull('hwchase17/openai-functions-agent')

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
from langchain.agents import AgentExecutor, create_openai_functions_agent

In [ ]:
agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=promt)

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

In [ ]:
llm_with_tools = llm.bind_tools(tools)
tool_map = {tool.name: tool for tool in tools}
def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls


chain = llm_with_tools | call_tools

In [ ]:
output = chain.invoke('what is 50+6, 3*4 and todays date')

In [ ]:
output

[{'name': 'add',
  'args': {'a': 50, 'b': 6},
  'id': 'call_3y4p',
  'type': 'tool_call',
  'output': 56},
 {'name': 'multiply',
  'args': {'a': 3, 'b': 4},
  'id': 'call_71e3',
  'type': 'tool_call',
  'output': 81},
 {'name': 'today_date',
  'args': {},
  'id': 'call_vtab',
  'type': 'tool_call',
  'output': '2024-12-24'}]

In [ ]:
for item in output:
  print(item['output'])

56
81
2024-12-24


In [ ]:
output[0]['output']

56